In [1]:
import sys

sys.path.insert(0, "../data_gen/")
sys.path.insert(0, "../layer/")
sys.path.insert(0,"../eval/")

import argparse
import os
import tensorflow as tf
from keras import backend as k
from hourglass import HourglassNet
from scipy.ndimage import gaussian_filter, maximum_filter
import numpy as np
import scipy.misc

Using TensorFlow backend.


In [7]:
imgfile = '../../tennis_in_crowd.jpg'
model_path = '../../trained_models/hg_epoch98.h5'
xnet = HourglassNet(num_classes=16, num_stacks=2, num_channels=256, inres=(256, 256),outres=(64, 64))
xnet.load_model(model_path)
out, scale = xnet.inference_file(imgfile)
kps = post_process_heatmap(out[0, :, :, :])

In [16]:
mkps = list()
for i, _kp in enumerate(kps):
    _conf = _kp[2]
    mkps.append((_kp[0] * scale[1] * 4, _kp[1] * scale[0] * 4, _conf))

In [17]:
mkps

[(208.40625, 352.6875, 0.64281076),
 (208.40625, 288.5625, 0.65934306),
 (248.484375, 216.421875, 0.4997584),
 (288.5625, 208.40625, 0.5844433),
 (312.609375, 288.5625, 0.6667665),
 (296.578125, 360.703125, 0.68440413),
 (264.515625, 216.421875, 0.60171103),
 (256.5, 104.203125, 0.30760473),
 (256.5, 96.1875, 0.5946255),
 (232.453125, 48.09375, 0.44429782),
 (264.515625, 184.359375, 0.53813297),
 (256.5, 168.328125, 0.3917832),
 (232.453125, 112.21875, 0.6085237),
 (280.546875, 96.1875, 0.23997575),
 (320.625, 120.234375, 0.23150209),
 (368.71875, 120.234375, 0.097175844)]

In [3]:
def normalize(imgdata, color_mean):
    '''
    :param imgdata: image in 0 ~ 255
    :return:  image from 0.0 to 1.0
    '''
    imgdata = imgdata / 255.0

    for i in range(imgdata.shape[-1]):
        imgdata[:, :, i] -= color_mean[i]

    return imgdata

In [4]:
inres = [256,256]
imgdata = scipy.misc.imread(imgfile)
orgshape = imgdata.shape
scale = (orgshape[0] * 1.0 / inres[0], orgshape[1] * 1.0 / inres[1])
imgdata = scipy.misc.imresize(imgdata, inres)
mean = np.array([0.4404, 0.4440, 0.4327], dtype=np.float)
imgdata = normalize(imgdata, mean)
input1 = imgdata[np.newaxis, :, :, :]

C:\Users\cxhrz\Anaconda3\envs\python36\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  
C:\Users\cxhrz\Anaconda3\envs\python36\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.3.0.
Use Pillow instead: ``numpy.array(Image.fromarray(arr).resize())``.
  """


In [145]:
orgshape

(513, 513, 3)

In [55]:
out = xnet.model.predict(input1)
out = out[1]

In [12]:
scale

(2.00390625, 2.00390625)

In [3]:
heatMap = out[0, :, :, :]

In [5]:
def post_process_heatmap(heatMap, kpConfidenceTh=0.2):
    kplst = list()
    for i in range(heatMap.shape[-1]):
        # ignore last channel, background channel
        _map = heatMap[:, :, i]
        _map = gaussian_filter(_map, sigma=0.5)
        _nmsPeaks = non_max_supression(_map, windowSize=3, threshold=1e-6)

        y, x = np.where(_nmsPeaks == _nmsPeaks.max())
        if len(x) > 0 and len(y) > 0:
            kplst.append((int(x[0]), int(y[0]), _nmsPeaks[y[0], x[0]]))
        else:
            kplst.append((0, 0, 0))
    return kplst


def non_max_supression(plain, windowSize=3, threshold=1e-6):
    # clear value less than threshold
    under_th_indices = plain < threshold
    #print(under_th_indices)
    plain[under_th_indices] = 0
    #print(plain)
    #print((plain == maximum_filter(plain, footprint=np.ones((windowSize, windowSize)))).dtype)
    return plain * (plain == maximum_filter(plain, footprint=np.ones((windowSize, windowSize))))

In [50]:
a = np.array([True,False])
b = np.array([2,3])
a*b

array([2, 0])

In [27]:
#print(help(scipy.ndimage.maximum_filter))

In [6]:
kps = list()
for i in range(heatMap.shape[-1]):
    _map = heatMap[:, :, i]
    #print(_map.shape)
    #print(_map)
    _map = gaussian_filter(_map, sigma=0.5)
    #_map2 = gfilter(_map,template)
    #_map3 = convolve(_map,gaussian_kernel())
    #print(_map.shape)
    _nmsPeaks = non_max_supression(_map, windowSize=3, threshold=1e-6)
    #print(tf.image.non_max_suppression(_map))
    #print(_nmsPeaks)
    print(_nmsPeaks.max())
    y, x = np.where(_nmsPeaks == _nmsPeaks.max())
    if len(x) > 0 and len(y) > 0:
        kps.append((int(x[0]), int(y[0]), _map[y[0], x[0]]))
    else:
        kps.append((0, 0, 0))

NameError: name 'heatMap' is not defined

In [12]:
kp_keys = ['r_ankle', 'r_knee', 'r_hip',
                'l_hip', 'l_knee', 'l_ankle',
                'plevis', 'thorax', 'upper_neck', 'head_top',
                'r_wrist', 'r_elbow', 'r_shoulder',
                'l_shoulder', 'l_elbow', 'l_wrist']
mkps = list()
for i, _kp in enumerate(kps):
    _conf = _kp[2]
    mkps.append((_kp[0] * scale[1] * 4, _kp[1] * scale[0] * 4, _conf))

In [ ]:
mkps

In [139]:
scale

(2.00390625, 2.00390625)

In [126]:
kps

[(26, 44, 0.64281076),
 (26, 36, 0.65934306),
 (31, 27, 0.4997584),
 (36, 26, 0.5844433),
 (39, 36, 0.6667665),
 (37, 45, 0.68440413),
 (33, 27, 0.60171103),
 (32, 13, 0.30760473),
 (32, 12, 0.5946255),
 (29, 6, 0.44429782),
 (33, 23, 0.53813297),
 (32, 21, 0.3917832),
 (29, 14, 0.6085237),
 (35, 12, 0.23997575),
 (40, 15, 0.23150209),
 (46, 15, 0.097175844)]

In [128]:
kps

[(26, 44, 0.64281076),
 (26, 36, 0.65934306),
 (31, 27, 0.4997584),
 (36, 26, 0.5844433),
 (39, 36, 0.6667665),
 (37, 45, 0.68440413),
 (33, 27, 0.60171103),
 (32, 13, 0.30760473),
 (32, 12, 0.5946255),
 (29, 6, 0.44429782),
 (33, 23, 0.53813297),
 (32, 21, 0.3917832),
 (29, 14, 0.6085237),
 (35, 12, 0.23997575),
 (40, 15, 0.23150209),
 (46, 15, 0.097175844)]

In [130]:
kps = list()
for i in range(heatMap.shape[-1]):
    _map = tf.reshape(heatMap[:,:,i],[-1])
    arg = k.argmax(_map,0)
    joints = tf.expand_dims(tf.stack([arg // tf.to_int64(shape[1]), arg % tf.to_int64(shape[1])], axis = -1), axis = 0)
    print(k.eval(joints))

[[44 26]]
[[36 26]]
[[27 30]]
[[26 36]]
[[36 39]]
[[45 37]]
[[27 33]]
[[13 32]]
[[12 32]]
[[ 6 29]]
[[23 33]]
[[21 32]]
[[14 29]]
[[12 35]]
[[15 40]]
[[15 46]]


In [121]:
2842//64

44

In [91]:
kps

[(26, 44, 0.64281076),
 (26, 36, 0.65934306),
 (31, 27, 0.4997584),
 (36, 26, 0.5844433),
 (39, 36, 0.6667665),
 (37, 45, 0.68440413),
 (33, 27, 0.60171103),
 (32, 13, 0.30760473),
 (32, 12, 0.5946255),
 (29, 6, 0.44429782),
 (33, 23, 0.53813297),
 (32, 21, 0.3917832),
 (29, 14, 0.6085237),
 (35, 12, 0.23997575),
 (40, 15, 0.23150209),
 (46, 15, 0.097175844)]

In [106]:
shape=heatMap.shape
if len(shape) == 3:
    resh = tf.reshape(heatMap[:,:,0], [-1])
elif len(shape) == 4:
    resh = tf.reshape(heatMap[-1,:,:,0], [-1])
arg =tf.argmax(resh,0)
k.eval(arg)

2842

In [102]:
joints = tf.expand_dims(tf.stack([arg // tf.to_int64(shape[1]), arg % tf.to_int64(shape[1])], axis = -1), axis = 0)
for i in range(1, shape[-1]):
    if len(shape) == 3:
        resh = tf.reshape(heatMap[:,:,i], [-1])
    elif len(shape) == 4:
        resh = tf.reshape(heatMap[-1,:,:,i], [-1])
    arg = tf.arg_max(resh,0)
    j = tf.expand_dims(tf.stack([arg // tf.to_int64(shape[1]), arg % tf.to_int64(shape[1])], axis = -1), axis = 0)
    joints = tf.concat([joints, j], axis = 0)

In [103]:
k.eval(joints)

array([[44, 26],
       [36, 26],
       [27, 30],
       [26, 36],
       [36, 39],
       [45, 37],
       [27, 33],
       [13, 32],
       [12, 32],
       [ 6, 29],
       [23, 33],
       [21, 32],
       [14, 29],
       [12, 35],
       [15, 40],
       [15, 46]], dtype=int64)

In [6]:
a = np.arange(50, step=2).reshape((5,5))
ga = gaussian_filter(a,sigma=0.5)
ta = gfilter(a,template)
np.sum(a)

600

In [4]:
import math
sigema = 0.5
radius = 2
def calc(x,y):
    res1=1/(2*math.pi*sigema*sigema)
    res2=math.exp(-(x*x+y*y)/(2*sigema*sigema))
    return res1*res2
#得到滤波模版
def template():
    sideLength=radius*2+1
    result = np.zeros((sideLength, sideLength))
    for i in range(sideLength):
        for j in range(sideLength):
               result[i,j]=calc(i-radius, j-radius)
    all=result.sum()
    a = result /all
    return a
def gfilter(arr, template):
    height=arr.shape[0]
    width=arr.shape[1]
    arr = np.pad(arr,((2,2)),'edge')
    newData=np.zeros((height, width)) 
    for i in range(height):
        for j in range(width):
            t = arr[i:i+2*radius+1,j:j+2*radius+1]
            a = np.multiply(t,template())
            newData[i,j] = a.sum()
    return newData
def gaussian_kernel(sigma=0.5, truncate=4.0):
    """
    Return Gaussian that truncates at the given number of standard deviations. 
    """

    sigma = float(sigma)
    radius = int(truncate * sigma + 0.5)

    x, y = np.mgrid[-radius:radius+1, -radius:radius+1]

    sigma = sigma**2
    G = np.exp(-0.5 * (np.power(x, 2) + np.power(y, 2)) / sigma)
    G = G / np.sum(G)

    return G
def convolve(src, F):
    H, W = src.shape
    HH, WW = F.shape

    #assert(H > HH)
    #assert(W > WW)
    hh = (HH-1) / 2
    ww = (WW-1)/ 2
    
    dst = np.zeros((H, W))

    src_pad = np.pad(src, (int(hh), int(ww)), 'edge')

    for i in range(H):
        for j in range(W):
            overlap = src_pad[i : i + HH, j: j + WW]
            average = np.sum(F * overlap)
            dst[i, j] = average    
    return dst

In [198]:
np.

2.0


array([[ 1.28374203,  3.07031275,  5.06978502,  7.06925729,  8.85582802],
       [10.21659566, 12.00316638, 14.00263865, 16.00211092, 17.78868165],
       [20.213957  , 22.00052773, 24.        , 25.99947227, 27.786043  ],
       [30.21131835, 31.99788908, 33.99736135, 35.99683362, 37.78340434],
       [39.14417198, 40.93074271, 42.93021498, 44.92968725, 46.71625797]])

In [111]:
print(help(np.multiply))

Help on ufunc object:

multiply = class ufunc(builtins.object)
 |  Functions that operate element by element on whole arrays.
 |  
 |  To see the documentation for a specific ufunc, use `info`.  For
 |  example, ``np.info(np.sin)``.  Because ufuncs are written in C
 |  (for speed) and linked into Python with NumPy's ufunc facility,
 |  Python's help() function finds this page whenever help() is called
 |  on a ufunc.
 |  
 |  A detailed explanation of ufuncs can be found in the docs for :ref:`ufuncs`.
 |  
 |  Calling ufuncs:
 |  ===============
 |  
 |  op(*x[, out], where=True, **kwargs)
 |  Apply `op` to the arguments `*x` elementwise, broadcasting the arguments.
 |  
 |  The broadcasting rules are:
 |  
 |  * Dimensions of length 1 may be prepended to either array.
 |  * Arrays may be repeated along dimensions of length 1.
 |  
 |  Parameters
 |  ----------
 |  *x : array_like
 |      Input arrays.
 |  out : ndarray, None, or tuple of ndarray and None, optional
 |      Alternate ar

In [109]:
template().dtype

dtype('float32')